In [1]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.coordinates as coord
import astropy.units as u
from astropy import time

from poliastro.bodies import Earth
from poliastro.twobody import Orbit
import poliastro

import urllib.request
import json
from collections import OrderedDict

In [2]:
url = 'http://www.celestrak.com/NORAD/elements/stations.txt'


response = urllib.request.urlopen(url)
data = response.read()      # a `bytes` object
text = data.decode('utf-8') # a `str`; this step can't be used if data is binary
data = text.splitlines()

for i in range(len(data)):
    if data[i].startswith('ISS (ZARYA)'):
        
        l1 = "1 25544U 98067A   19190.67737078  .00000405  00000-0  14728-4 0  9990"
        l2 = "2 25544  51.6431 249.6881 0007257 122.7057  10.0638 15.50965501178747"
        
        l1 = data[i+1].split()
        l2 = data[i+2].split()
        
        
        time_vector = l1[3]        
        year = int("20"+time_vector[0:2])
        ydayfraction = float(time_vector[2:])
        
                
        Inclination    = float(l2[2]) * u.deg
        RA_of_node     = float(l2[3]) * u.deg
        Eccentricity   = float("0."+l2[4]) * u.one
        Arg_of_perigee = float(l2[5]) * u.deg
        Mean_anomaly   = float(l2[6]) * u.deg
        Rev_Per_Day    = (float(l2[7])  / u.day).to(1/u.s)
        semi_major_Axis = Earth.k**(1/3) / (2 * np.pi * Rev_Per_Day)**(2/3)
        

        print("ISS TLE from celestrack.")
        print(data[i+1])
        print(data[i+2])
        print("\n")
        
        message = ("\tInclination    : {}\n"
                   "\tRA of node     : {}\n"
                   "\tEccentricity   : {}\n"
                   "\tArg of Perigee : {}\n"
                   "\tMean_Anomaly   : {}\n"
                   "\tSemi-major Axis: {}").format(Inclination, 
                                                   RA_of_node, 
                                                   Eccentricity,
                                                   Arg_of_perigee, 
                                                   Mean_anomaly, 
                                                   semi_major_Axis)
    
        print("ISS Orbital Elements from TLE\n\n" + message ) 
              
            
Epoch_time = time.Time("{}-01-01 00:00".format(year))
Epoch_time += (ydayfraction -1) * u.day

print("TLE at: ", Epoch_time.yday, " that means", Epoch_time)

ZARYA = poliastro.twobody.Orbit.from_classical(Earth,
                                              semi_major_Axis,
                                              Eccentricity,
                                              Inclination,
                                              RA_of_node,
                                              Arg_of_perigee,
                                              Mean_anomaly,
                                              Epoch_time)

Tinit = time.Time("2019-07-11 1:59")
DeltaT = Tinit - Epoch_time
#print(Tnow,"\n", Epoch_time,"\n", DeltaT)

ISSnow = ZARYA.propagate( DeltaT )

ISS TLE from celestrack.
1 25544U 98067A   19203.81086311  .00000606  00000-0  18099-4 0  9996
2 25544  51.6423 184.5274 0006740 168.1171 264.4057 15.50995519180787


ISS Orbital Elements from TLE

	Inclination    : 51.6423 deg
	RA of node     : 184.5274 deg
	Eccentricity   : 0.000674
	Arg of Perigee : 168.1171 deg
	Mean_Anomaly   : 264.4057 deg
	Semi-major Axis: 6791955.199068729 m
TLE at:  2019:203:19:27:38.573  that means 2019-07-22 19:27:38.573


The state of ISS (r,v) in the Non Rotating Frame is:

In [3]:
ISSnow.r

<Quantity [ 2297.94235016, -3859.86431577,  5091.3153819 ] km>

In [4]:
ISSnow.v

<Quantity [ 7.19386905,  1.96774643, -1.76114734] km / s>

Now we load the ground stations info.

In [5]:
with open('ground_stations.json') as json_file:
    data = json.load(json_file)

In [6]:
GS = []
gs = []

for  k in sorted(data.keys()):
    GS.append(data[k])

In [7]:
for k in GS:
    gs.append(coord.EarthLocation( k["gs_lon"], k["gs_lat"], k["gs_alt"]))
    

Their location in the rotating system is:

In [8]:
gs

[<EarthLocation (3946293.3013581, 809282.50717543, 4928547.87338264) m>,
 <EarthLocation (4157409.29535264, 672080.10355092, 4774350.40490151) m>,
 <EarthLocation (3899925.15472251, 855534.85353504, 4957331.36135458) m>,
 <EarthLocation (3857116.33132939, 516634.17159883, 5036460.26932419) m>]

In [9]:
gs[0].to_geodetic()

GeodeticLocation(lon=<Longitude 11.5892 deg>, lat=<Latitude 50.9271 deg>, height=<Quantity 143. m>)

In [10]:
gs_gcrs = []
gs_vel_gcrs = []

In [11]:
for k in gs:
    gs_gcrs.append( k.get_itrs(Tinit) )
    gs_vel_gcrs.append( k.get_gcrs_posvel(obstime= Tinit))

In [12]:
gs_vel_gcrs

[(<CartesianRepresentation (x, y, z) in m
      (3490465.92481309, -2027129.61147224, 4922006.3289872)>,
  <CartesianRepresentation (x, y, z) in m / s
      (147.81360806, 253.85855918, -0.27098031)>),
 (<CartesianRepresentation (x, y, z) in m
      (3556109.03683172, -2270124.01989083, 4767681.92682764)>,
  <CartesianRepresentation (x, y, z) in m / s
      (165.5331088, 258.66636823, -0.30399438)>),
 (<CartesianRepresentation (x, y, z) in m
      (3486782.27598969, -1961744.19604804, 4950797.85424236)>,
  <CartesianRepresentation (x, y, z) in m / s
      (143.045498, 253.5860977, -0.26209798)>),
 (<CartesianRepresentation (x, y, z) in m
      (3229128.27183646, -2185853.36158231, 5030404.05841471)>,
  <CartesianRepresentation (x, y, z) in m / s
      (159.38778464, 234.78677187, -0.29297606)>)]

In [13]:
a = gs[0].get_gcrs_posvel(obstime=Tinit)[0]

In [14]:
a.xyz

<Quantity [ 3490465.92481309, -2027129.61147224,  4922006.3289872 ] m>

In [15]:
ISSnow.r - a.xyz

<Quantity [-1192.52357466, -1832.7347043 ,   169.30905291] km>

In [16]:
ISSnow.v - gs[0].get_gcrs_posvel(obstime=Tinit)[1].xyz

<Quantity [ 7.04605544,  1.71388787, -1.76087636] km / s>

In [17]:
ISSnow.v

<Quantity [ 7.19386905,  1.96774643, -1.76114734] km / s>

In [18]:
gs[0].get_gcrs_posvel(obstime=Tinit)[1].xyz

<Quantity [147.81360806, 253.85855918,  -0.27098031] m / s>

In [19]:
R_iss_gs0 = ISSnow.r - a.xyz

In [20]:
R_iss_gs0

<Quantity [-1192.52357466, -1832.7347043 ,   169.30905291] km>

In [21]:
dist = 0
for k in R_iss_gs0:
    dist += k**2
dist = (dist)**0.5

In [22]:
dist

<Quantity 2193.10157719 km>

In [23]:
R_iss_gs0 / dist

<Quantity [-0.54376121, -0.83568163,  0.07720073]>

In [24]:
R_iss_gs0_unitary = R_iss_gs0 / dist

In [25]:
V_iss_gs0 = ISSnow.v - gs[0].get_gcrs_posvel(obstime=Tinit)[1].xyz

In [26]:
V_iss_gs0.dot(R_iss_gs0_unitary)

<Quantity -5.39957722 km / s>

Ok,

We have the ISS close to Germany on Thursday 2019-07-11 at 2:05 GMT

According to https://spotthestation.nasa.gov/ the station will pass over germany in 5 minutes.

The time window will be set for 8 minutes, sampling each 10 seconds.

For testing purposes, all stations are sampling synchronously.

In [27]:
Tinit

<Time object: scale='utc' format='iso' value=2019-07-11 01:59:00.000>

In [28]:
42 / 6

7.0

In [29]:
T_minutos = 9

deltat = 10 * u.s

intervalos = T_minutos * 6

In [30]:
ISS_evolution = []
T = []
GS0_loc = []
GS1_loc = []
GS2_loc = []
GS3_loc = []


for i in range( intervalos ):
    T.append(Tinit + deltat * i)
    ISS_evolution.append( ISSnow.propagate(deltat * i) )
    GS0_loc.append( gs[0].get_gcrs_posvel(obstime = Tinit + deltat*i) )
    GS1_loc.append( gs[1].get_gcrs_posvel(obstime = Tinit + deltat*i) )
    GS2_loc.append( gs[2].get_gcrs_posvel(obstime = Tinit + deltat*i) )
    GS3_loc.append( gs[3].get_gcrs_posvel(obstime = Tinit + deltat*i) )

In [31]:
ISSnow.r

<Quantity [ 2297.94235016, -3859.86431577,  5091.3153819 ] km>

In [32]:
def norm( v ):
    norm = 0
    for k in v:
        norm += k**2
    norm = norm**0.5
    
    return norm

# Data for JENA

In [46]:
for o, gs, t in zip(ISS_evolution, GS0_loc, T):
    iss_rel_to_gs = o.r - gs[0].xyz
    V_iss_rel_to_gs = o.v - gs[1].xyz
    V = V_iss_rel_to_gs
    
    dist = norm(iss_rel_to_gs)
    
    iss_rel_to_gs_unitary = iss_rel_to_gs / dist
    R_u = iss_rel_to_gs_unitary
    
    
    
    print('{}  {:06.3f}  {:06.3f}'.format(t,dist, V.dot(R_u)))

2019-07-11 01:59:00.000  2193.102 km  -5.400 km / s
2019-07-11 01:59:10.000  2139.572 km  -5.305 km / s
2019-07-11 01:59:20.000  2087.028 km  -5.203 km / s
2019-07-11 01:59:30.000  2035.550 km  -5.091 km / s
2019-07-11 01:59:40.000  1985.230 km  -4.971 km / s
2019-07-11 01:59:50.000  1936.163 km  -4.841 km / s
2019-07-11 02:00:00.000  1888.452 km  -4.700 km / s
2019-07-11 02:00:10.000  1842.209 km  -4.547 km / s
2019-07-11 02:00:20.000  1797.552 km  -4.382 km / s
2019-07-11 02:00:30.000  1754.609 km  -4.204 km / s
2019-07-11 02:00:40.000  1713.513 km  -4.013 km / s
2019-07-11 02:00:50.000  1674.404 km  -3.807 km / s
2019-07-11 02:01:00.000  1637.431 km  -3.586 km / s
2019-07-11 02:01:10.000  1602.745 km  -3.349 km / s
2019-07-11 02:01:20.000  1570.502 km  -3.097 km / s
2019-07-11 02:01:30.000  1540.859 km  -2.829 km / s
2019-07-11 02:01:40.000  1513.972 km  -2.546 km / s
2019-07-11 02:01:50.000  1489.993 km  -2.248 km / s
2019-07-11 02:02:00.000  1469.068 km  -1.935 km / s
2019-07-11 0

# Data for all stations.

In [34]:
GroundStationLocations = [GS0_loc, GS1_loc, GS2_loc, GS3_loc]

In [47]:
for GS_loc in GroundStationLocations:
    for o, gs, t in zip(ISS_evolution, GS_loc, T):
        iss_rel_to_gs = o.r - gs[0].xyz
        V_iss_rel_to_gs = o.v - gs[1].xyz
        V = V_iss_rel_to_gs

        dist = norm(iss_rel_to_gs)

        iss_rel_to_gs_unitary = iss_rel_to_gs / dist
        R_u = iss_rel_to_gs_unitary



        print('{}  {:06.3f}  {:06.3f}'.format(t,dist, V.dot(R_u)))
        
    print("\n\n")

2019-07-11 01:59:00.000  2193.102 km  -5.400 km / s
2019-07-11 01:59:10.000  2139.572 km  -5.305 km / s
2019-07-11 01:59:20.000  2087.028 km  -5.203 km / s
2019-07-11 01:59:30.000  2035.550 km  -5.091 km / s
2019-07-11 01:59:40.000  1985.230 km  -4.971 km / s
2019-07-11 01:59:50.000  1936.163 km  -4.841 km / s
2019-07-11 02:00:00.000  1888.452 km  -4.700 km / s
2019-07-11 02:00:10.000  1842.209 km  -4.547 km / s
2019-07-11 02:00:20.000  1797.552 km  -4.382 km / s
2019-07-11 02:00:30.000  1754.609 km  -4.204 km / s
2019-07-11 02:00:40.000  1713.513 km  -4.013 km / s
2019-07-11 02:00:50.000  1674.404 km  -3.807 km / s
2019-07-11 02:01:00.000  1637.431 km  -3.586 km / s
2019-07-11 02:01:10.000  1602.745 km  -3.349 km / s
2019-07-11 02:01:20.000  1570.502 km  -3.097 km / s
2019-07-11 02:01:30.000  1540.859 km  -2.829 km / s
2019-07-11 02:01:40.000  1513.972 km  -2.546 km / s
2019-07-11 02:01:50.000  1489.993 km  -2.248 km / s
2019-07-11 02:02:00.000  1469.068 km  -1.935 km / s
2019-07-11 0

2019-07-11 02:06:30.000  2294.919 km  05.672 km / s
2019-07-11 02:06:40.000  2351.991 km  05.741 km / s
2019-07-11 02:06:50.000  2409.721 km  05.804 km / s
2019-07-11 02:07:00.000  2468.056 km  05.862 km / s
2019-07-11 02:07:10.000  2526.949 km  05.916 km / s
2019-07-11 02:07:20.000  2586.353 km  05.965 km / s
2019-07-11 02:07:30.000  2646.227 km  06.010 km / s
2019-07-11 02:07:40.000  2706.533 km  06.051 km / s
2019-07-11 02:07:50.000  2767.235 km  06.089 km / s





With the data, we can create the JSON files.

# Redshift Formula

Taking into acocunt Transverse Relativistic Redshift

#### Doppler Measurement and Compensation in Mobile Satellite Communications Systems

Quingchou Liu


$f = f_c \left( 1 + \frac{v(t)}{c} \cos \left(\alpha(t)\right) \right)$

With $\alpha$ as the angle between the satellite's velocity and the direction of propagation.

In [36]:
c = 299792.458 * u.km / u.s

def redshift(f_0, Vradial ):
    
    return f_0 * (1 + Vradial / c)

# Data to JSON

In [48]:
comment = "Testing data. ISS Passing over Germany on 2019-07-11 1:59."

file_name = "jena_iss_test.json"
gs_count  = 1
sat_count = 1

gs_id     = "GS_JENA_1"
lat       = GS[0]['gs_lat']
lon       = GS[0]['gs_lon']
alt       = GS[0]['gs_alt']
monitored = "ISS"

sat_id    = 25544
f_c       = 630.0 * (10**6) / u.s # ISS telemetry MHz


In [49]:
data_stream = []

for o, gs, t in zip(ISS_evolution, GS0_loc, T):
    iss_rel_to_gs = o.r - gs[0].xyz
    V_iss_rel_to_gs = o.v - gs[1].xyz
    V = V_iss_rel_to_gs
    
    dist = norm(iss_rel_to_gs)
    
    iss_rel_to_gs_unitary = iss_rel_to_gs / dist
    R_u = iss_rel_to_gs_unitary
    
    
    item = OrderedDict([
            ('time_unix'   , '{:06.3f}'.format(t.unix)), 
            ('time_gnss'   , None),
            ('f_m'         , '{:06.2f}'.format(redshift(f_c, V.dot(R_u)).value)),
            ('gs_distance' , '{:06.2f}'.format(dist.value))
                       ])
    
    data_stream.append(item)
    
print(data_stream[0])

OrderedDict([('time_unix', '1562810340.000'), ('time_gnss', None), ('f_m', '629988653.04'), ('gs_distance', '2193.10')])


In [50]:
data_to_parse = OrderedDict([('__comment'    , comment),
                 ('meta_data',  
                  OrderedDict([('file_name'   , file_name),
                               ('gs_count'    , gs_count), 
                               ('sat_count'   , sat_count)])),
                 ('ground_station',
                  OrderedDict([                  
                             ('gs_id'       , gs_id),
                             ('lat'         , lat),
                             ('lon'         , lon),
                             ('alt'         , alt),
                             ('monitored'   , monitored)])),
                 ('satellites',[
                  OrderedDict([
                         ('sat_id'      , sat_id),
                         ('f_c'         , f_c.value),
                         ('data_stream' , data_stream)]) 
                 ]) 
                ])

In [51]:
print(json.dumps(data_to_parse, indent=4, sort_keys=False))

{
    "__comment": "Testing data. ISS Passing over Germany on 2019-07-11 1:59.",
    "meta_data": {
        "file_name": "jena_iss_test.json",
        "gs_count": 1,
        "sat_count": 1
    },
    "ground_station": {
        "gs_id": "GS_JENA_1",
        "lat": 50.9271,
        "lon": 11.5892,
        "alt": 143,
        "monitored": "ISS"
    },
    "satellites": [
        {
            "sat_id": 25544,
            "f_c": 630000000.0,
            "data_stream": [
                {
                    "time_unix": "1562810340.000",
                    "time_gnss": null,
                    "f_m": "629988653.04",
                    "gs_distance": "2193.10"
                },
                {
                    "time_unix": "1562810350.000",
                    "time_gnss": null,
                    "f_m": "629988851.74",
                    "gs_distance": "2139.57"
                },
                {
                    "time_unix": "1562810360.000",
                    "time_gns